In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from IPython.display import display, clear_output
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label
from ipywidgets import interactive_output, interact, fixed, FloatLogSlider, HBox, Layout, Output, VBox

from magpie import magpie

In [ ]:
labels = ['K0y', 'Kx0', 'KLy', 'KxL', 'R0y', 'Rx0', 'RLy', 'RxL']

sliders =  {label:FloatLogSlider(
    value=0.0, base=10, min=0, max=15,
    description=label,
    continuous_update=True,
    orientation='vertical' if 'x' in label else 'horizontal',
    readout_format='.1e',
    layout=Layout(flex='1 1 1', width='auto')
) for label in labels}


In [ ]:
%matplotlib inline

Lx = 1.1; Ly = 1.1; Lz = 5e-3
ldim = [Lx, Ly, Lz]  # -- plate dimensions [x, y, z] in metres
E = 9.0e+9           # -- Young's mod [Pa]
rho = 8765           # -- density [kg/m^3]
nu = 0.3             # -- poisson's ratio
Nmodes = 4           # -- number of modes to compute
h = np.sqrt(Lx * Ly) * 0.05   # -- grid spacing
BCs = np.ones((4, 2)) * 1e15 # -- elastic constants around the edges

chladni_plot = Output();


def plot_mode_shapes(K0y, Kx0, KLy, KxL, R0y, Rx0, RLy, RxL):
    BCs = np.array([[Kx0,Rx0],[K0y,R0y],[KxL,RxL],[KLy,RLy]], dtype=np.float64)
    with chladni_plot:
        
        Q, Om, N, biharm = magpie(rho, E, nu, ldim, h, BCs, Nmodes)
        for m in range(Nmodes):
            mode_shapes = [np.real(np.reshape(Q[:, m], [N['x'], N['y']])) for m in range(Nmodes)]

        clear_output(wait=True)
        
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)

        Z = abs(mode_shapes[3])
        chladni = plt.pcolormesh(Z.T, cmap='copper_r', shading='gouraud')
        ax.set_axis_off()
        chladni.set_clim(0.01, 0.05)
        
        plt.plot()        
        plt.show()

In [ ]:
interactive_output(plot_mode_shapes, sliders)
plot_mode_shapes(*[s.value for s in sliders.values()])

display(
    VBox([VBox([sliders['KLy'],sliders['RLy']]),
          HBox([sliders['Kx0'],sliders['Rx0'],chladni_plot, sliders['KxL'],sliders['RxL']]),
          VBox([sliders['K0y'],sliders['R0y']])])
)